In [1]:
import os
from langchain_openai import ChatOpenAI
from langgraph.graph import Graph
from langgraph.prebuilt import create_react_agent
import gradio as gr
from datetime import datetime

# Import tools
from tools.calculator import CalculatorTool
from tools.city_recommender import CityRecommendationTool
from tools.email_tool import EmailItineraryTool
from tools.hotel_search import HotelSearchTool
from tools.user_preferences import UserPreferencesTool
from tools.flight_search import FlightSearchTool
from tools.map_tool import MapPlottingTool
from tools.attractions_tool import CityAttractionsTool
from city_vector_db import CityVectorDB

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
def initialize_tools():

    vector_db = CityVectorDB()
    vector_db.load("cities_vector_db")

    tools = [
        UserPreferencesTool(),
        FlightSearchTool(api_key=os.getenv('SERPAPI_API_KEY')),
        MapPlottingTool(),
        EmailItineraryTool(
            smtp_user="h.belagur1980@gmail.com",
            smtp_password=os.getenv('GMAIL_PASSWORD')
        ),
        CityAttractionsTool(),
        HotelSearchTool(api_key=os.getenv('SERPAPI_API_KEY')),
        CalculatorTool(), 
        CityRecommendationTool(vector_db=vector_db)
    ]
    return tools

In [3]:
tools = initialize_tools()

In [19]:
system_prompt = f"""You are a helpful travel assistant named Fred.

Follow these steps in order, do not send an email until the conversation is over or the user explicitly asks for it:

1. INITIAL GREETING:
   - Get the user's name
   - Using the name, check if they have stored preferences using the user_preferences_tool
   
2. PREFERENCES (only if needed):
   - If no preferences exist, ask about their travel preferences
   - Store their preferences using user_preferences_tool
   
3. CITY RECOMMENDATION:
   - Use city_recommendation_tool EXACTLY ONCE to suggest cities
   - Ask the user if they'd like to search for flights to any of the recommended cities
   
4. FLIGHT SEARCH (when user selects a destination):
   - Use the user's hometown as the departure_airport, if it is not known, get it from the user
   - Ask for specific travel dates (departure and return, if return is not given, get the duration of the trip)
   - Use flight_search_tool with IATA codes
   - Format: {{
       'departure_airport': '[IATA code]',
       'arrival_airport': '[IATA code]',
       'departure_date': 'YYYY-MM-DD',
       'return_date': 'YYYY-MM-DD'  # Optional for one-way flights
   }}

5. NEXT STEPS:
   - After showing flight results, ask if they want to:
     a) Search flights for different dates
     b) Search for a different destination

5. ATTRACTIONS:
   - Once the user selects a destination and a flight, use the city_attractions_tool to get the top attractions in the destination. Make sure you match the correct formats for the tool

6. MAP:
   - Using the attractions retrieved, use the map_plotting_tool to create a map of the trip
   
7. HOTEL SEARCH (when user selects a destination and a flight):
   - Look for hotels in the destination for the dates of the flight (departure and return date)

8. CALCULATOR:
   - If the user asks about the cost of a trip, use the calculator_tool to sum up the selected flight and hotel costs and also provide the breakdown of the costs   

9. END:
   - Once the user is done with the conversation, thank them, get their email and end the conversation telling them that you will email them the detials
   
Remember that today's date is {datetime.now().strftime('%Y-%m-%d')}. Only search for future dates.
Do not repeat city recommendations unless explicitly asked by the user.
"""

In [16]:
def create_agent(system_prompt):
    model = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.getenv('OPENAI_API_KEY'))
    tools = initialize_tools()
    return create_react_agent(model, tools=tools, state_modifier=system_prompt, debug=True)

In [17]:
agent = create_agent(system_prompt)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
def respond(message, history):
    """Process user message and return agent response"""
    # Convert history to the format expected by the agent
    messages = []
    for human, assistant in history:
        messages.append(("user", human))
        if assistant:  # Skip None responses
            messages.append(("assistant", assistant))
    
    # Add current message
    messages.append(("user", message))
    
    # Process through graph and get response
    inputs = {"messages": messages}
    
    try:
        for s in agent.stream(inputs, stream_mode="values"):
            print(s)
            message = s["messages"][-1]
            # print(message)
            if isinstance(message, tuple):
                # Extract just the content from the tuple
                response = message[1]
            else:
                # Extract content from AIMessage or other message types
                response = message.content if hasattr(message, 'content') else str(message)
            yield response
    except Exception as e:
        yield f"Error: {str(e)}\nPlease try again or refresh the page."

# Update the Gradio interface
demo = gr.ChatInterface(
    respond,
    title="Fred the Travel Assistant",
    description="Ask Fred about your next trip!",
    examples=["Hi, I'm planning a trip"],
)

demo.launch()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:243: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', "Hi, I'm planning a trip")]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', "Hi, I'm planning a trip")]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content="Hi, I'm planning a trip", additional_kwargs={}, response_metadata={}, id='367f0d75-cc7d-4a7f-8e37-6e91067aaba9')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content="Hi, I'm planning a trip", additional_kwargs={}, response_metadata={}, id='367f0d75-cc7d-4a7f-8e37-6e91067aaba9')],
 'remaining_steps': 24}
{'messages': [HumanMessage(content="Hi, I'm planning a trip", additional_kwargs={}, response_metadata={}, id='367f0d75-cc7d-4a7f-8e37-6e91067aaba9')]}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content="Great to hear that you're planning a trip! I can